In [57]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
import os 

In [90]:
# expanded commonsense dataset 
commonsense_data=pd.read_excel('/Users/juniorcedrictonga/RA_MBZUAI/Cultural_Commonsense_Knowledge_Graph/results/monolingual_extension.xlsx',sheet_name='Indonesia')
#commonsense_data.duplicated()
#commonsense_data[commonsense_data.duplicated()]
commonsense_data=commonsense_data[['event','knowledge']]
duplicates = commonsense_data[commonsense_data.duplicated(keep=False)]
duplicates


,event,knowledge
16,choosing a quick lunch option,x might prefer a grab-and-go meal like nasi bu...
20,a family is having dinner together,they likely engage in conversation
58,eat durian in Indonesia,x might want to try it with friends or family ...
74,selecting keripik singkong to take home,the person plans to share it with family
108,x discusses dowry with future in-laws,x wants to understand expectations
...,...,...
6830,the player likely wants to try again,they might feel a sense of friendly competitiv...
6841,x wants to socialize with other attendees,x will receive and share community news
6865,participating in a Pesantren Ramadan,o wants to share their own religious experienc...
6870,participating in a Pesantren Ramadan,o wants to share their own religious experienc...


In [74]:
def get_valeur(df, element):
    return df.loc[df['event'] == element, 'knowledge'].values[0]

valeur = get_valeur(commonsense_data, '其他人会回应并一起举杯')  
print(valeur) 

475       这时可以发表祝酒词
476    大家会一起享用餐前的小菜
Name: knowledge, dtype: object


In [66]:
england_sub_topics=['breakfast','dinner','traditional foods and beverages','cutlery','cooking ware', 'fruit', 'food souvenirs', 'snacks',
               'wedding location', 'wedding dowry', 'traditions before marriage', 'traditions when getting married', 'traditions after marriage', 
               'men’s wedding clothes', 'women’s wedding clothes', 'songs and activities during the wedding', 'invited guests at a wedding', 'gift brought to weddings', 'food at a wedding',
                'eating habit', 'greetings habits', 'financial habits (saving, debit/credit)', 'punctuality habits', 'cleanliness habits', 'shower time habits', 'transportation habits', 
                'popular sports','musical instruments', 'folks songs', 'traditional dances', 'use of art at certain events', 'poetry or similar literature',
               'morning activities', 'afternoon activities', 'evening activities', 'Leisure and relaxation activities', 'Household activities (cleaning, home management)',
                 'relationships within the main family', 'relationships in the extended family', 'relations with society/neighbors', 'clan/descendant system',
                 'traditions during pregnancy', 'traditions after birth', 'how to care for a newborn baby', 'how to care for toddlers', 'how to care for teenagers', 'parents and children interactions as adults', 'when death occurs', 'the process of dealing with a corpse', 'traditions after the body is buried', 'the clothes of the mourners','inheritance matters',
                'traditions before religious holidays', 'traditions leading up to religious holidays', 'traditions during religious holidays', 'traditions after holidays',
               'traditional game types','regular religious activities', 'mystical things', 'traditional ceremonies', 'lifestyle', 'self care', 'traditional medicine', 'traditional sayings']
len(england_sub_topics)

64

In [59]:
#cleaning 
clean_commonsense_data=commonsense_data[['event','knowledge','relation','llm_result','location','sub_topic']].drop_duplicates()
clean_commonsense_data=clean_commonsense_data[clean_commonsense_data['event'] != clean_commonsense_data['knowledge']]
clean_commonsense_data = clean_commonsense_data[~clean_commonsense_data["event"].isin(["oNext", "xNext"]) & ~clean_commonsense_data["knowledge"].isin(["oNext", "xNext"])]
clean_commonsense_data

,event,knowledge,relation,llm_result,location,sub_topic
0,喝茶,可能想为茶友准备点心,xNext,如果喝茶，那么可能想为茶友准备点心。,China,传统的食物和饮料
1,准备一些福建安溪铁观音作为纪念品,在当地茶庄品尝不同种类是个不错的选择,xNext,如果准备一些福建安溪铁观音作为纪念品，那么在当地茶庄品尝不同种类是个不错的选择。,China,可食用的纪念品
2,穿上婚礼旗袍,通常会搭配传统的刺绣鞋,xNext,如果穿上婚礼旗袍，那么通常会搭配传统的刺绣鞋。,China,女士的婚礼服饰
3,新人切婚礼蛋糕,人们期待分享这份甜蜜,oNext,如果新人切婚礼蛋糕，那么人们期待分享这份甜蜜。,China,婚礼上的食物
4,在寺庙前举办婚礼仪式,可能希望遵循传统的拜佛仪式以祈求祝福,xNext,如果在寺庙前举办婚礼仪式，那么可能希望遵循传统的拜佛仪式以祈求祝福。,China,婚礼地点
...,...,...,...,...,...,...
472,如果席主可能会表示下次再聚,其他宾客可能会提醒互相保持联系,oNext,如果席主可能会表示下次再聚，那么其他宾客可能会提醒互相保持联系,China,食具
473,可能想在敬酒时说一些祝福的话,可能想保持微笑与大家互动,xNext,如果可能想在敬酒时说一些祝福的话，那么可能想保持微笑与大家互动,China,饮食习惯
474,可能想在敬酒时说一些祝福的话,在场的其他人可能会举杯回应并附和祝福,oNext,如果可能想在敬酒时说一些祝福的话，那么在场的其他人可能会举杯回应并附和祝福,China,饮食习惯
475,其他人会回应并一起举杯,这时可以发表祝酒词,xNext,如果其他人会回应并一起举杯，那么这时可以发表祝酒词,China,饮食习惯


## Sub-graph extraction

In [26]:
# DFS approach to extract paths even if the subgraph is cyclic but too consuming
#all_paths = []
#paths_with_relations = []
#for source in G_sub.nodes:
    #    stack = [(source, [source])]  # Pile pour DFS : (noeud courant, chemin actuel)
        
     #   while stack:
    #        current_node, path = stack.pop()
            
            # Ajouter un chemin s'il est valide
     #       if len(path) >= 3:
      #          relations = [
      #              G_sub.get_edge_data(path[i], path[i + 1])['relation']
      #              for i in range(len(path) - 1)
      #          ]
      #          paths_with_relations.append((path, relations))
      #          all_paths.append(path)
      #      
            # Étendre le chemin en explorant les voisins
      #      for neighbor in G_sub.neighbors(current_node):
       #         if len(path) + 1 > len(G_sub):  # Prévenir les boucles infinies dans des graphes cycliques
      #              continue
        #        stack.append((neighbor, path + [neighbor]))

    # Structurer les données des chemins extraits
    #paths_data = [
       # {
    #        'path': " -> ".join(path),
      #      'relations': " - ".join(relations),
      #      'sub_topic': G_sub.nodes[path[0]].get('sub_topic', 'N/A'),  
     #       'path_len': len(path)
    #    }
    #    for path, relations in paths_with_relations
    #    ]


In [20]:
def export_to_excel(file_path, dataframe, sheet_name):
    if os.path.exists(file_path):
        with pd.ExcelWriter(file_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
            dataframe.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
            dataframe.to_excel(writer, sheet_name=sheet_name, index=False)



In [ ]:
df = clean_commonsense_data  # Input DataFrame
G = nx.DiGraph()

# Construct the main graph
for _, row in df.iterrows():
    G.add_edge(row['event'], row['knowledge'], relation=row['relation'])

sub_topics = df['sub_topic'].unique()
global_paths_df = pd.DataFrame(columns=['path', 'relations', 'sub_topic', 'path_len'])
subtopic_stats = {}

# Helper function to break cycles
def remove_cycles(graph):
    while True:
        try:
            cycle = next(nx.simple_cycles(graph))
            # Remove the last edge in the cycle
            graph.remove_edge(cycle[-1], cycle[0])
        except StopIteration:
            break

# Iterate over sub-topics
for sub_topic in sub_topics:
    df_sub = df[df['sub_topic'] == sub_topic]
    G_sub = nx.DiGraph()
    
    # Construct the subgraph
    for _, row in df_sub.iterrows():
        G_sub.add_edge(row['event'], row['knowledge'], relation=row['relation'])
    
    # Remove cycles from the subgraph
    remove_cycles(G_sub)
    
    # Collect statistics for the subgraph
    num_nodes = G_sub.number_of_nodes()
    num_edges = G_sub.number_of_edges()
    degree_distribution = dict(G_sub.degree())
    average_degree = sum(degree_distribution.values()) / num_nodes if num_nodes > 0 else 0
    central_nodes = sorted(degree_distribution.items(), key=lambda x: x[1], reverse=True)[:5]
    subtopic_stats[sub_topic] = {
        'num_nodes': num_nodes,
        'num_edges': num_edges,
        'average_degree': average_degree,
        'central_nodes': central_nodes
    }
    
    # Extract paths of length >= 3
    all_simple_paths = []
    paths_with_relations = []
    longest_path = nx.dag_longest_path(G_sub)
    print("\nLongest path : {}".format(longest_path))
    for source in G_sub.nodes:
        for target in G_sub.nodes:
            if source != target:
                paths = list(nx.all_simple_paths(G_sub, source=source, target=target))
                filtered_paths = [path for path in paths if len(path) >= 3]
                all_simple_paths.extend(filtered_paths)
                
                # Extract relations for each path
                for path in filtered_paths:
                    relations = [
                        G_sub.get_edge_data(path[i], path[i + 1])['relation']
                        for i in range(len(path) - 1)
                    ]
                    paths_with_relations.append((path, relations))
    
    # Prepare path data for DataFrame
    paths_data = [
        {
            'path': " -> ".join(path),
            'relations': " - ".join(relations),
            'sub_topic': sub_topic,
            'path_len': len(path)
        }
        for path, relations in paths_with_relations
    ]
    global_paths_df = pd.concat([global_paths_df, pd.DataFrame(paths_data)], ignore_index=True)
    
    # Plot the subgraph
    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(G_sub, seed=42)
    nx.draw(G_sub, pos, with_labels=True, node_size=50, edge_color="blue", alpha=0.7, font_size=8)
    edge_labels = nx.get_edge_attributes(G_sub, 'relation')
    nx.draw_networkx_edge_labels(G_sub, pos, edge_labels=edge_labels, font_size=8, font_color="red")
    plt.title(f"Graph per sub-topic: {sub_topic}")
    plt.show()

# Print statistics
print("\nStatistics by sub-topics:")
for sub_topic, stats in subtopic_stats.items():
    print(f"\nSub-topic: {sub_topic}")
    print(f"- Number of nodes: {stats['num_nodes']}")
    print(f"- Number of edges: {stats['num_edges']}")
    print(f"- Mean degree: {stats['average_degree']:.2f}")
    print(f"- Most central nodes (by degree):")
    for node, degree in stats['central_nodes']:
        print(f"  - {node}: {degree} connections")


In [29]:
global_paths_df

,path,relations,sub_topic,path_len
0,喝茶 -> 为茶找到合适的茶具 -> 选择适合的茶叶 -> 品尝茶的风味 -> 可能想为茶友...,xNext - xNext - xNext - xNext,传统的食物和饮料,5
1,喝茶 -> 为茶找到合适的茶具 -> 选择适合的茶叶,xNext - xNext,传统的食物和饮料,3
2,喝茶 -> 为茶找到合适的茶具 -> 选择适合的茶叶 -> 品尝茶的风味,xNext - xNext - xNext,传统的食物和饮料,4
3,喝茶 -> 可能想为茶友准备点心 -> 准备一些传统的中式糕点,xNext - xNext,传统的食物和饮料,3
4,喝茶 -> 可能想为茶友准备点心 -> 选择适合搭配茶的甜咸兼备的点心种类 -> 考虑如绿豆...,xNext - xNext - xNext - xNext,传统的食物和饮料,5
...,...,...,...,...
2421,准备的酒通常包括白酒或红酒 -> 还需要准备好酒杯 -> 其他人会回应并一起举杯,xNext - xNext,饮食习惯,3
2422,准备的酒通常包括白酒或红酒 -> 还需要准备好酒杯 -> 其他人会回应并一起举杯 -> 这时...,xNext - xNext - xNext,饮食习惯,4
2423,准备的酒通常包括白酒或红酒 -> 还需要准备好酒杯 -> 其他人会回应并一起举杯 -> 大家...,xNext - xNext - oNext,饮食习惯,4
2424,还需要准备好酒杯 -> 其他人会回应并一起举杯 -> 这时可以发表祝酒词,xNext - xNext,饮食习惯,3


In [13]:
global_paths_df['sub_topic'].unique().shape

(18,)

In [ ]:
# with attribut on paths

df=clean_commonsense_data
G = nx.DiGraph()

for _, row in df.iterrows():
    G.add_edge(row['event'], row['knowledge'], relation=row['relation'])

sub_topics = df['sub_topic'].unique()
global_paths_df = pd.DataFrame(columns=['path_with_relations', 'relations', 'sub_topic', 'path_len'])

subtopic_stats = {}

for sub_topic in sub_topics:
    df_sub = df[df['sub_topic'] == sub_topic]
    G_sub = nx.DiGraph()
    for _, row in df_sub.iterrows():
        G_sub.add_edge(row['event'], row['knowledge'], relation=row['relation'])
    
    # stat per subtopics
    num_nodes = G_sub.number_of_nodes()
    num_edges = G_sub.number_of_edges()
    degree_distribution = dict(G_sub.degree())  
    average_degree = sum(degree_distribution.values()) / num_nodes if num_nodes > 0 else 0
    
    central_nodes = sorted(degree_distribution.items(), key=lambda x: x[1], reverse=True)[:5]
    subtopic_stats[sub_topic] = {
        'num_nodes': num_nodes,
        'num_edges': num_edges,
        'average_degree': average_degree,
        'central_nodes': central_nodes
    }
   # path extraction with len >= 3
    if nx.is_directed_acyclic_graph(G_sub):
        all_simple_paths = []
        paths_with_relations = []
        longest_path = nx.dag_longest_path(G_sub)
        print("\nLongest path : {}".format(longest_path))
        for source in G_sub.nodes:
            for target in G_sub.nodes:
                if source != target:
                    # Extraction of path
                    paths = list(nx.all_simple_paths(G_sub, source=source, target=target, cutoff=None))
                    filtered_paths = [path for path in paths if len(path) >= 3]
                    all_simple_paths.extend(filtered_paths)

                    # Extraction of relations
                    for path in filtered_paths:
                        relations = [
                            G_sub.get_edge_data(path[i], path[i + 1])['relation']
                            for i in range(len(path) - 1)
                        ]
                        paths_with_relations.append((path, relations))
        paths_data = []
        for path, relations in paths_with_relations:
             #path_with_relations = path[0]
             path_with_relations = ""
             for i, relation in enumerate(relations):
                 path_with_relations += f"{path[i]}---{relation} ---> {path[i + 1]}"+"\n"
             paths_data.append({
            'path_with_relations': path_with_relations.strip(), 
            'sub_topic': sub_topic, 
            'relations': " - ".join(relations),  
            'path_len':len(path)  
        })
        global_paths_df = pd.concat([global_paths_df, pd.DataFrame(paths_data)], ignore_index=True)

            #print("\n path with (len >= 3) with relation:")
            #for path, relations in paths_with_relations:
            #    print(f"Path: {' -> '.join(path)}")
            #   print(f"Relations: {' -> '.join(relations)}")
    else:
        print("\n graph is cyclic")

    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(G_sub, seed=42) 
    nx.draw(G_sub, pos, with_labels=True, node_size=50, edge_color="blue", alpha=0.7, font_size=8)
    edge_labels = nx.get_edge_attributes(G_sub, 'relation')
    nx.draw_networkx_edge_labels(G_sub, pos, edge_labels=edge_labels, font_size=8, font_color="red")

    plt.title(f"Graph per sub-topics : {sub_topic}")
    plt.show()


print("\nStatistics by sub-topics :")
for sub_topic, stats in subtopic_stats.items():
    print(f"\nSub-topic : {sub_topic}")
    print(f"- Number of nodes : {stats['num_nodes']}")
    print(f"- Number of edge: {stats['num_edges']}")
    print(f"- mean degree : {stats['average_degree']:.2f}")
    print(f"- Most central node (per degre) :")
    for node, degree in stats['central_nodes']:
        print(f"  - {node}: {degree} connexions")

In [61]:
global_paths_df

,path_with_relations,relations,sub_topic,path_len
0,准备一些福建安溪铁观音作为纪念品---xNext ---> 了解铁观音的历史和制作工艺\n了...,xNext - xNext - xNext - xNext,可食用的纪念品,5
1,准备一些福建安溪铁观音作为纪念品---xNext ---> 了解铁观音的历史和制作工艺\n了...,xNext - xNext,可食用的纪念品,3
2,准备一些福建安溪铁观音作为纪念品---xNext ---> 了解铁观音的历史和制作工艺\n了...,xNext - xNext - xNext,可食用的纪念品,4
3,准备一些福建安溪铁观音作为纪念品---xNext ---> 在当地茶庄品尝不同种类是个不错的...,xNext - xNext,可食用的纪念品,3
4,准备一些福建安溪铁观音作为纪念品---xNext ---> 在当地茶庄品尝不同种类是个不错的...,xNext - xNext - xNext - xNext - xNext,可食用的纪念品,6
...,...,...,...,...
1986,准备的酒通常包括白酒或红酒---xNext ---> 还需要准备好酒杯\n还需要准备好酒杯-...,xNext - xNext,饮食习惯,3
1987,准备的酒通常包括白酒或红酒---xNext ---> 还需要准备好酒杯\n还需要准备好酒杯-...,xNext - xNext - xNext,饮食习惯,4
1988,准备的酒通常包括白酒或红酒---xNext ---> 还需要准备好酒杯\n还需要准备好酒杯-...,xNext - xNext - oNext,饮食习惯,4
1989,还需要准备好酒杯---xNext ---> 其他人会回应并一起举杯\n其他人会回应并一起举杯...,xNext - xNext,饮食习惯,3


In [ ]:
# with cycle 

df=clean_commonsense_data
G = nx.DiGraph()

for _, row in df.iterrows():
    G.add_edge(row['event'], row['knowledge'], relation=row['relation'])

sub_topics = df['sub_topic'].unique()
global_paths_df = pd.DataFrame(columns=['path', 'relations', 'sub_topic', 'path_len'])

subtopic_stats = {}

for sub_topic in sub_topics:
    df_sub = df[df['sub_topic'] == sub_topic]
    G_sub = nx.DiGraph()
    for _, row in df_sub.iterrows():
        G_sub.add_edge(row['event'], row['knowledge'], relation=row['relation'])
    
    # stat per subtopics
    num_nodes = G_sub.number_of_nodes()
    num_edges = G_sub.number_of_edges()
    degree_distribution = dict(G_sub.degree())  
    average_degree = sum(degree_distribution.values()) / num_nodes if num_nodes > 0 else 0
    
    central_nodes = sorted(degree_distribution.items(), key=lambda x: x[1], reverse=True)[:5]
    subtopic_stats[sub_topic] = {
        'num_nodes': num_nodes,
        'num_edges': num_edges,
        'average_degree': average_degree,
        'central_nodes': central_nodes
    }
   # path extraction with len >= 3
    if nx.is_directed_acyclic_graph(G_sub):
        all_simple_paths = []
        paths_with_relations = []
        longest_path = nx.dag_longest_path(G_sub)
        print("\nLongest path : {}".format(longest_path))
        for source in G_sub.nodes:
            for target in G_sub.nodes:
                if source != target:
                    # Extraction of path
                    paths = list(nx.all_simple_paths(G_sub, source=source, target=target, cutoff=None))
                    filtered_paths = [path for path in paths if len(path) >= 3]
                    all_simple_paths.extend(filtered_paths)

                    # Extraction of relations
                    for path in filtered_paths:
                        relations = [
                            G_sub.get_edge_data(path[i], path[i + 1])['relation']
                            for i in range(len(path) - 1)
                        ]
                        paths_with_relations.append((path, relations))
        paths_data = [
            {   'path': " -> ".join(path),
                'relations': " - ".join(relations),
                'sub_topic': sub_topic,   
                'path_len':len(path)
               
            }
            for path, relations in paths_with_relations
        ]
        global_paths_df = pd.concat([global_paths_df, pd.DataFrame(paths_data)], ignore_index=True)

            #print("\n path with (len >= 3) with relation:")
            #for path, relations in paths_with_relations:
            #    print(f"Path: {' -> '.join(path)}")
            #   print(f"Relations: {' -> '.join(relations)}")
    else:
        print("\n graph is cyclic")

    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(G_sub, seed=42) 
    nx.draw(G_sub, pos, with_labels=True, node_size=50, edge_color="blue", alpha=0.7, font_size=8)
    edge_labels = nx.get_edge_attributes(G_sub, 'relation')
    nx.draw_networkx_edge_labels(G_sub, pos, edge_labels=edge_labels, font_size=8, font_color="red")

    plt.title(f"Graph per sub-topics : {sub_topic}")
    plt.show()


print("\nStatistics by sub-topics :")
for sub_topic, stats in subtopic_stats.items():
    print(f"\nSub-topic : {sub_topic}")
    print(f"- Number of nodes : {stats['num_nodes']}")
    print(f"- Number of edge: {stats['num_edges']}")
    print(f"- mean degree : {stats['average_degree']:.2f}")
    print(f"- Most central node (per degre) :")
    for node, degree in stats['central_nodes']:
        print(f"  - {node}: {degree} connexions")

In [54]:
global_paths_df

,path_with_relations,relations,sub_topic,path_len
0,准备一些福建安溪铁观音作为纪念品---xNext ---> 了解铁观音的历史和制作工艺\n了...,xNext - xNext - xNext - xNext,可食用的纪念品,5
1,准备一些福建安溪铁观音作为纪念品---xNext ---> 了解铁观音的历史和制作工艺\n了...,xNext - xNext,可食用的纪念品,3
2,准备一些福建安溪铁观音作为纪念品---xNext ---> 了解铁观音的历史和制作工艺\n了...,xNext - xNext - xNext,可食用的纪念品,4
3,准备一些福建安溪铁观音作为纪念品---xNext ---> 在当地茶庄品尝不同种类是个不错的...,xNext - xNext,可食用的纪念品,3
4,准备一些福建安溪铁观音作为纪念品---xNext ---> 在当地茶庄品尝不同种类是个不错的...,xNext - xNext - xNext - xNext - xNext,可食用的纪念品,6
...,...,...,...,...
1986,准备的酒通常包括白酒或红酒---xNext ---> 还需要准备好酒杯\n还需要准备好酒杯-...,xNext - xNext,饮食习惯,3
1987,准备的酒通常包括白酒或红酒---xNext ---> 还需要准备好酒杯\n还需要准备好酒杯-...,xNext - xNext - xNext,饮食习惯,4
1988,准备的酒通常包括白酒或红酒---xNext ---> 还需要准备好酒杯\n还需要准备好酒杯-...,xNext - xNext - oNext,饮食习惯,4
1989,还需要准备好酒杯---xNext ---> 其他人会回应并一起举杯\n其他人会回应并一起举杯...,xNext - xNext,饮食习惯,3


In [62]:
# export
file_path='/Users/juniorcedrictonga/RA_MBZUAI/Cultural_Commonsense_Knowledge_Graph/multilingual_path_per_countries_subtopics_test_new_with_atttrib.xlsx'
export_to_excel(file_path,global_paths_df,'China')
#global_paths_df.to_excel('/Users/juniorcedrictonga/RA_MBZUAI/Cultural_Commonsense_Knowledge_Graph/path_per_countries_subtopics_test_new.xlsx',sheet_name='China',index=False)

In [63]:
total_samples = 400
min_per_subtopic = max(1, total_samples // global_paths_df['sub_topic'].nunique())
grouped = global_paths_df.groupby('sub_topic', group_keys=False)
samples = grouped.apply(lambda x: x.sample(n=min(len(x), min_per_subtopic), random_state=42))
remaining = total_samples - len(samples)
if remaining > 0:
    additional_samples = global_paths_df.drop(samples.index).sample(n=remaining, random_state=42)
    samples = pd.concat([samples, additional_samples])

samples


,path_with_relations,relations,sub_topic,path_len
128,与当地茶农交流喝茶的习惯---xNext ---> 在当地茶庄品尝不同种类是个不错的选择\n...,xNext - oNext - xNext,可食用的纪念品,4
45,准备一些福建安溪铁观音作为纪念品---xNext ---> 了解铁观音的历史和制作工艺\n了...,xNext - xNext - xNext - xNext - oNext - oNext,可食用的纪念品,7
134,了解各种茶叶的风味特色---xNext ---> 询问茶庄工作人员推荐不同茶叶适合的口味\n...,xNext - xNext - xNext,可食用的纪念品,4
156,前往推荐的茶叶店进行购买---xNext ---> 购买一些品尝到的高品质铁观音带回家\n购...,xNext - xNext,可食用的纪念品,3
90,参观安溪铁观音的茶园和茶厂---xNext ---> 与当地茶农交流喝茶的习惯\n与当地茶农...,xNext - xNext - xNext,可食用的纪念品,4
...,...,...,...,...
1536,获得年终奖---xNext ---> 考虑如何合理投资和储蓄\n考虑如何合理投资和储蓄---...,xNext - xNext - xNext - xNext - xNext - xNext,财务习惯（储蓄、借记/贷记）,7
336,蛋糕上会放一个小模型新郎新娘---xNext ---> 新人共同切蛋糕\n新人共同切蛋糕--...,xNext - oNext - oNext,婚礼上的食物,4
34,准备一些福建安溪铁观音作为纪念品---xNext ---> 了解铁观音的历史和制作工艺\n了...,xNext - xNext - xNext - xNext - xNext - xNext ...,可食用的纪念品,10
1673,握手---xNext ---> 对接触中动态把控\n对接触中动态把控---xNext ---...,xNext - xNext,问候习惯,3


In [64]:
#export
file_path='/Users/juniorcedrictonga/RA_MBZUAI/Cultural_Commonsense_Knowledge_Graph/multilingual_300_path_per_countries_subtopics_with_atttrib.xlsx'
export_to_excel(file_path,samples,'China')
#samples.to_excel('/Users/juniorcedrictonga/RA_MBZUAI/Cultural_Commonsense_Knowledge_Graph/300_path_per_countries_subtopics.xlsx',sheet_name='China')

# Comparaison with others CKG : MAMGO and CANDLE

In [11]:
model=SentenceTransformer("all-MiniLM-L6-v2")
threshold=0.8

## 1. Mango

In [ ]:
file_path='/Users/juniorcedrictonga/RA_MBZUAI/mango_dataset_v1.jsonl'
mango_data=pd.read_json(file_path,lines=True)
mango_data

In [ ]:
def extract_matching_assertinons_with_mango(mango_data, commonsense_data,model, threshold, use_just_indonesian_data_in_mango=True):
    if use_just_indonesian_data_in_mango:
        # Extract Indonesian data in Mango
        terms = ["Indonesia", "Southeast Asia", "Asia-Pacific","Global South","Indonesian"]
        regex_pattern = '|'.join(terms)
        mango_data['contains_relevant_terms'] = mango_data['culture'].str.contains(regex_pattern, case=False, na=False)
        relevant_rows = mango_data[mango_data['contains_relevant_terms']]
    else:
        relevant_rows=mango_data


    # find overlapping with our dataset and relevant rows of mango
    relevant_rows_embedd=model.encode(relevant_rows['assertion'].tolist())
    commonsense_data_embedd=model.encode(commonsense_data['llm_result'].tolist())
    similarities=model.similarity(commonsense_data_embedd,relevant_rows_embedd)
    results=[]
    for i, idx in tqdm(enumerate(commonsense_data.index)): 
        for j in range (len(relevant_rows)): 
            similarity = similarities[i][j].item()
            if similarity > threshold:
                results.append({
                    'mango_assertion': relevant_rows['assertion'].iloc[j], 
                    'llm_result': commonsense_data.at[idx,'llm_result'],  
                    'relation': commonsense_data.at[idx, 'relation'],  
                    'similarity': similarity,
                    'mango_concept': relevant_rows['concept'].iloc[j],
                    'mango_culture': relevant_rows['culture'].iloc[j],
                    'sub_topic': commonsense_data.at[idx, 'sub_topic'],
                    'event': commonsense_data.at[idx, 'event'],
                    'knowledge': commonsense_data.at[idx,'knowledge'],
                    'commonsense_data_index':idx
                })
    
    matches_df = pd.DataFrame(results)
    return relevant_rows, matches_df



In [ ]:
idonesian_rows_in_mango, mango_matches_data=extract_matching_assertinons_with_mango(mango_data,
                                                                                    clean_commonsense_data,model,threshold,
                                                                                    use_just_indonesian_data_in_mango=True)

In [ ]:
idonesian_rows_in_mango

In [ ]:
mango_matches_data

In [ ]:
# number of sample of mango that overlapped with our data is 
mango_matches_data['mango_assertion'].unique().shape

## 2. Candle

In [ ]:
file_path='/Users/juniorcedrictonga/RA_MBZUAI/candle_dataset_v1.jsonl'
candle_data=pd.read_json(file_path,lines=True)
candle_data

In [ ]:
# find overlapping with our dataset and relevant rows of candle
commonsense_data=clean_commonsense_data
candle_rows_embedd=model.encode(candle_data['assertion'].tolist())
commonsense_data_embedd=model.encode(commonsense_data['llm_result'].tolist())
similarities=model.similarity(commonsense_data_embedd,candle_rows_embedd)
results=[]
for i, idx in tqdm(enumerate(commonsense_data.index)): 
    for j in range (len(candle_data)): 
        similarity = similarities[i][j].item()
        if similarity > threshold:
            results.append({
                'candle_assertion': candle_data['assertion'].iloc[j], 
                'llm_result': commonsense_data.at[idx,'llm_result'],  
                'relation': commonsense_data.at[idx, 'relation'],  
                'similarity': similarity,
                'candle_facet': candle_data['facet'].iloc[j],
                'candle_subject': candle_data['subject'].iloc[j],
                'candle_domain':candle_data['domain'].iloc[j],
                'candle_concepts':candle_data['concepts'].iloc[j],
                'sub_topic': commonsense_data.at[idx, 'sub_topic'],
                'event': commonsense_data.at[idx, 'event'],
                'knowledge': commonsense_data.at[idx,'knowledge'],
                'commonsense_data_index':idx
            })

candle_matches_df = pd.DataFrame(results)

In [ ]:
candle_matches_df

In [ ]:
# number of sample of candle that overlappes with our data is 
candle_matches_df['candle_assertion'].unique().shape